In [1]:
import pickle
import random
from datasets import load_dataset

/home/alex/miniconda3/envs/elk/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load from ../elk-reporters/serene-meitner/transfer_eval/layer_0026_propositions.pkl
proposition_results = pickle.load(open("../elk-reporters/serene-meitner/transfer_eval/layer_0026_propositions.pkl", "rb"))

/home/alex/miniconda3/envs/elk/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
proposition_results["cal_preds"][1]r

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)

In [9]:
# the keys are cal_preds, raw_preds, test_labels, test_text_inputs
# print out the size of each of the lists and their sublists
# check the shape of cal_preds
print(proposition_results["cal_preds"].shape)

# check the shape of raw_preds
print(proposition_results["raw_preds"].shape)

# check the size of test_labels
print(proposition_results["test_labels"].shape)

# check the size of test_text_inputs
print(len(proposition_results["test_text_inputs"]))
print(len(proposition_results["test_text_inputs"][0]))

torch.Size([250, 12])
torch.Size([250, 12])
torch.Size([250])
250
12


In [15]:
def flatten_python_list(lst):
    # flatten the python list
    return [item for sublist in lst for item in sublist]

In [26]:
labels = proposition_results["test_labels"]
cal_preds = proposition_results["cal_preds"]
raw_preds = proposition_results["raw_preds"]
broadcast_labels = labels.repeat_interleave(cal_preds.shape[1]).float()
cal_preds_flattened = cal_preds.flatten().eq(broadcast_labels)
cal_acc = cal_preds_flattened.float().mean()
raw_preds_flattened = raw_preds.flatten().eq(broadcast_labels)
raw_acc = raw_preds_flattened.float().mean()
inputs_flattened = flatten_python_list(proposition_results["test_text_inputs"])

In [30]:
# select the elments of inputs_flattened that that correspond to the indices of cal_preds_flattened that are True
cal_correct_inputs = [inputs_flattened[i] for i in range(len(cal_preds_flattened)) if cal_preds_flattened[i]]

# select the elments of inputs_flattened that that correspond to the indices of cal_preds_flattened that are False
cal_incorrect_inputs = [inputs_flattened[i] for i in range(len(cal_preds_flattened)) if not cal_preds_flattened[i]]

In [39]:
# randomly sample from the list of correct inputs
random.sample(cal_correct_inputs, 10)

[['Zamia vazquezii is a species of plant in the Zamiaceae family. It is endemic to Mexico. It is threatened by habitat loss.\nWhat label best describes this paragraph, choice 1: Plant, or choice 2: Building\nPlant',
  'Zamia vazquezii is a species of plant in the Zamiaceae family. It is endemic to Mexico. It is threatened by habitat loss.\nWhat label best describes this paragraph, choice 1: Building, or choice 2: Plant\nBuilding'],
 ['"Murgaš (Ub)", given a list of categories: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work, what category does the title belong to? Animal',
  '"Murgaš (Ub)", given a list of categories: company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work, what category does the title belong to? Village'],
 ["Consider the following example: '

a lot of correct examples from dbpedia are like. Title: X is a Y. and then Y is exactly one of the category choices. Or, the title literally has the category word in it.

In [38]:
# randomly sample from the list of incorrect inputs
random.sample(cal_incorrect_inputs, 10)

[["Thomas Dehler (14 December 1897 – 21 July 1967) was a German politician. He was the Federal Republic of Germany's first Minister of Justice (1949–1953) and chairman of Free Democratic Party (1954–1957).\nChoice 1: Building. Choice 2: Office Holder.Between choice 1 and choice 2, the topic of this example is Building",
  "Thomas Dehler (14 December 1897 – 21 July 1967) was a German politician. He was the Federal Republic of Germany's first Minister of Justice (1949–1953) and chairman of Free Democratic Party (1954–1957).\nChoice 1: Office Holder. Choice 2: Building.Between choice 1 and choice 2, the topic of this example is Office Holder"],
 ['Pick one category for the following text. The options are - company, educational institution, artist, athlete, office holder, mean of transportation, building, natural place, village, animal, plant, album, film or written work. Kangaroo River (Shoalhaven) -  The Kangaroo River is a perennial river of the Shoalhaven catchment located in the South

Idea: generate factual statements that are very simple using a knowledge base (if possible, this should be comparable to the triples used by ROME)
- We want to see if there is a correlation between the popularity of subjects (via their wikipedia pageview count) and the performance of CCS/VINC on those subjects
- By making the prompts factual statements, and by making them as simple as possible, we isolate the "factual knowledge" of the model

In [2]:
dataset = load_dataset("NeelNanda/counterfact-tracing")

Found cached dataset parquet (/home/alex/.cache/huggingface/datasets/NeelNanda___parquet/NeelNanda--counterfact-tracing-39c4f800d46af5cf/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 498.97it/s]


In [3]:
dataset['train']

Dataset({
    features: ['relation', 'relation_prefix', 'relation_suffix', 'prompt', 'relation_id', 'target_false_id', 'target_true_id', 'target_true', 'target_false', 'subject'],
    num_rows: 21919
})

In [9]:
# we want to find out how many different subjects there are
subjects = {}
for i in range(len(dataset['train'])):
    subjects[dataset['train'][i]['subject']] = subjects.get(dataset['train'][i]['subject'], []) + [i]
    

To search for a wikipedia article, make a request like this:
https://en.wikipedia.org/w/api.php?action=opensearch&search=university%20of%20waterloo&limit=1&format=json

To get the monthly pageview count for that wikipedia article, make a request like this:
https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/user/Barack_Obama/monthly/20220401/20230401

In [14]:
dataset['train'][subjects['3 lbs']]

{'relation': ['{} was released on'],
 'relation_prefix': [''],
 'relation_suffix': ['{} was released on'],
 'prompt': ['3 lbs was released on'],
 'relation_id': ['P449'],
 'target_false_id': ['Q13974'],
 'target_true_id': ['Q43380'],
 'target_true': [' CBS'],
 'target_false': [' NBC'],
 'subject': ['3 lbs']}

In [2]:
# load the counterfact dataset originally released by the ROME authors
import json
with open("./counterfact.json", "rb") as f:
    counterfact_dataset = json.load(f)

In [3]:
from datasets import Dataset
# create a dataset with the same data, but with the columns as the keys
counterfact_dict = {key: [item[key] for item in counterfact_dataset] for key in counterfact_dataset[0]}
hf_counterfact = Dataset.from_dict(counterfact_dict)

In [4]:
# sample 10% of the counterfact dataset and make it the test split
random.seed(42)
test_indices = random.sample(range(len(hf_counterfact)), int(len(hf_counterfact) * 0.1))
test_split = hf_counterfact.select(test_indices)
train_split = hf_counterfact.select([i for i in range(len(hf_counterfact)) if i not in test_indices])

In [5]:
# make the dataset dict
from datasets import DatasetDict
hf_dataset = DatasetDict({'train': train_split, 'test': test_split})

In [9]:
counterfact_dataset[0]

{'case_id': 0,
 'pararel_idx': 2796,
 'requested_rewrite': {'prompt': 'The mother tongue of {} is',
  'relation_id': 'P103',
  'target_new': {'str': 'English', 'id': 'Q1860'},
  'target_true': {'str': 'French', 'id': 'Q150'},
  'subject': 'Danielle Darrieux'},
 'paraphrase_prompts': ['Shayna does this and Yossel goes still and dies. Danielle Darrieux, a native',
  'An album was recorded for Capitol Nashville but never released. Danielle Darrieux spoke the language'],
 'neighborhood_prompts': ['The mother tongue of Léon Blum is',
  'The native language of Montesquieu is',
  'François Bayrou, a native',
  'The native language of Raymond Barre is',
  'Michel Rocard is a native speaker of',
  'Jacques Chaban-Delmas is a native speaker of',
  'The native language of François Bayrou is',
  'Maurice Genevoix, speaker of',
  'The mother tongue of François Bayrou is',
  'Melchior de Vogüé, speaker of'],
 'attribute_prompts': ['J.\xa0R.\xa0R. Tolkien is a native speaker of',
  'The mother tongue

In [6]:
from datasets import ClassLabel
stripped_dataset = []
for each in counterfact_dataset:
    target_true = each["requested_rewrite"]["target_true"]["str"]
    target_false = each["requested_rewrite"]["target_new"]["str"]
    labels = [target_false, target_true]
    for i, answer in enumerate(labels):
        row = {
            "subject": each["requested_rewrite"]["subject"],
            "proposition": each["requested_rewrite"]["prompt"].format(each["requested_rewrite"]["subject"]) + " " + answer,
            "label": i,
            "case_id": each["case_id"],
        }
        stripped_dataset.append(row.copy())
        #row["proposition"] = each["paraphrase_prompts"][0] + " " + answer
        #stripped_dataset.append(row.copy())
        #row["proposition"] = each["paraphrase_prompts"][1] + " " + answer
        #stripped_dataset.append(row.copy())
        # assume that each row has only two 

In [7]:
from datasets import Dataset, Features, Value
import random
from datasets import DatasetDict
# create a dataset with the same data, but with the columns as the keys
stripped_dict = {key: [item[key] for item in stripped_dataset] for key in stripped_dataset[0]}
hf_counterfact_stripped = Dataset.from_dict(stripped_dict, features=Features({
    'subject': Value(dtype='string', id=None),
    'proposition': Value(dtype='string', id=None),
    'label': ClassLabel(num_classes=2, names=['False', 'True']),
    'case_id': Value(dtype='int64', id=None)}))
# sample 10% of the counterfact dataset and make it the test split
random.seed(42)
test_indices = random.sample(range(len(hf_counterfact_stripped)), int(len(hf_counterfact_stripped) * 0.1))
test_split_stripped = hf_counterfact_stripped.select(test_indices)
train_split_stripped = hf_counterfact_stripped.select([i for i in range(len(hf_counterfact_stripped)) if i not in test_indices])
hf_dataset_stripped = DatasetDict({'train': train_split_stripped, 'test': test_split_stripped})

In [13]:
hf_dataset_stripped['train'][10]

{'subject': 'Thomas Joannes Stieltjes',
 'proposition': 'The mother tongue of Thomas Joannes Stieltjes is English',
 'label': 0,
 'case_id': 5}

In [14]:
hf_dataset_stripped.push_to_hub("azhx/counterfact-easy")

Pushing split train to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:01<00:00,  1.95s/it]
Pushing split test to the Hub.
Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:00<00:00,  1.08it/s]


In [59]:
from datasets import load_dataset
imdb = load_dataset("imdb")

Found cached dataset imdb (/home/alex/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 382.12it/s]


In [68]:
[each for each in imdb['train'] if each['label'] == 1]

[{'text': 'Zentropa has much in common with The Third Man, another noir-like film set among the rubble of postwar Europe. Like TTM, there is much inventive camera work. There is an innocent American who gets emotionally involved with a woman he doesn\'t really understand, and whose naivety is all the more striking in contrast with the natives.<br /><br />But I\'d have to say that The Third Man has a more well-crafted storyline. Zentropa is a bit disjointed in this respect. Perhaps this is intentional: it is presented as a dream/nightmare, and making it too coherent would spoil the effect. <br /><br />This movie is unrelentingly grim--"noir" in more than one sense; one never sees the sun shine. Grim, but intriguing, and frightening.',
  'label': 1},
 {'text': "Zentropa is the most original movie I've seen in years. If you like unique thrillers that are influenced by film noir, then this is just the right cure for all of those Hollywood summer blockbusters clogging the theaters these day

Why doesn't ELK work for this counterfactual dataset that I just made?

In [16]:
import pickle
proposition_results = pickle.load(open("../elk-reporters/zealous-mclaren/transfer_eval/layer_0027_propositions.pkl", "rb"))

In [32]:
proposition_results['cal_preds'].shape

torch.Size([250, 6])

In [6]:
len(proposition_results["test_labels"])

250